In [10]:
import cv2
import numpy as np
import time
from random import randint
class try_match:
    def __init__(self, frame):
        self.tb = cv2.imread('images/fb.png', cv2.IMREAD_GRAYSCALE)
        self.frame = frame

    def matching_Compute(self):
        orb = cv2.ORB_create()
        kp1, des1 = orb.detectAndCompute(self.tb, None)
        kp2, des2 = orb.detectAndCompute(self.frame, None)

        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        matches=bf.match(des1,des2)
        matches=sorted(matches, key=lambda x:x.distance)
        matching_result = cv2.drawMatches(self.tb, kp1, self.frame, kp2, matches[:10], None)
        print(matches[:10])
        return matching_result

cap=cv2.VideoCapture(1)

#Get width and height of video

w=cap.get(3)
h=cap.get(4)
frameArea=h*w
areaTH=frameArea/400
#Lines
line_up=int(2*(h/10))
# line_down=int(3*(h/5))

up_limit=int(1*(h/10))
# down_limit=int(4*(h/5))

# print("Black line y:",str(line_down))
# print("Blue line y:",str(line_up))
# line_down_color=(255,0,0)
line_up_color=(255,0,0)
# pt1 =  [0, line_down]
# pt2 =  [w, line_down]
# pts_L1 = np.array([pt1,pt2], np.int32)
# pts_L1 = pts_L1.reshape((-1,1,2))
pt3 =  [0, line_up]
pt4 =  [w, line_up]
pts_L2 = np.array([pt3,pt4], np.int32)
pts_L2 = pts_L2.reshape((-1,1,2))

pt5 =  [0, up_limit]
pt6 =  [w, up_limit]
pts_L3 = np.array([pt5,pt6], np.int32)
pts_L3 = pts_L3.reshape((-1,1,2))
# pt7 =  [0, down_limit]
# pt8 =  [w, down_limit]
# pts_L4 = np.array([pt7,pt8], np.int32)
# pts_L4 = pts_L4.reshape((-1,1,2))

#Background Subtractor
fgbg=cv2.createBackgroundSubtractorMOG2(detectShadows=True)

#Kernals
kernalOp = np.ones((3,3),np.uint8)
kernalOp2 = np.ones((5,5),np.uint8)
kernalCl = np.ones((11,11),np.uint)


font = cv2.FONT_HERSHEY_SIMPLEX
# cars = []
# max_p_age = 5
# pid = 1

old_x = 0
old_y = 0
old_w = 0
old_h = 0
old_cx = 0
old_cy = 0

count = 0
insertingFlag=False
while(cap.isOpened()):
    ret,frame=cap.read()
    # for i in cars:
        # i.age_one()
    fgmask=fgbg.apply(frame)
    fgmask2=fgbg.apply(frame)

    if ret==True:

        #Binarization
        ret,imBin=cv2.threshold(fgmask,200,255,cv2.THRESH_BINARY)
        ret,imBin2=cv2.threshold(fgmask2,200,255,cv2.THRESH_BINARY)
        #OPening i.e First Erode the dilate
        mask=cv2.morphologyEx(imBin,cv2.MORPH_OPEN,kernalOp)
        mask2=cv2.morphologyEx(imBin2,cv2.MORPH_CLOSE,kernalOp)
        # cv2.imshow("background E", mask)

        mask=cv2.morphologyEx(imBin,cv2.MORPH_CLOSE,kernalCl)
        mask2=cv2.morphologyEx(imBin,cv2.MORPH_CLOSE,kernalCl)
        # cv2.imshow("background D", mask)

        #Closing i.e First Dilate then Erode
        # mask=cv2.morphologyEx(mask,cv2.MORPH_CLOSE,kernalCl)
        # mask2=cv2.morphologyEx(mask2,cv2.MORPH_CLOSE,kernalCl)
        # cv2.imshow("background ED", mask)
        
        #Find Contours
        _, contours0,hierarchy=cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
        if len(contours0) != 0:
            cnt = max(contours0, key = cv2.contourArea) # pega somente o contour de area maxima
            area=cv2.contourArea(cnt)
            if area>areaTH:
                ####Tracking######
                m=cv2.moments(cnt)
                cx=int(m['m10']/m['m00'])
                cy=int(m['m01']/m['m00'])
                x,y,w,h=cv2.boundingRect(cnt)
                # print(x,':',y,':',h,':',w)
                new=True
                if cv2.waitKey(1)&0xff==ord('p'):
                    cv2.imwrite("frames/frame%d.jpg" % count, frame)
                    count += 1
                if y+h > up_limit:
                    if insertingFlag == True:
                        p=try_match(frame)
                        # cv2.imshow("match",p.matching_Compute())
                        # cv2.waitKey(0)
                        # cv2.destroyAllWindows()

                        # if cv2.waitKey(1)&0xff==ord('p'):
                        #     cv2.imwrite("frames/frame%d.jpg" % count, frame)
                        #     count += 1
                        # print('going up')
                        # if i.going_UP(line_down,line_up)==True:
                        # print('if ',old_y+old_h, ' > ', line_up, 'and ', y+h, ' <= ', line_up)

                        if old_y+old_h > line_up and y+h <= line_up:
                            insertingFlag=False
                            # print('insertingFlag=False')
                    elif insertingFlag == False:   
                        # print('going down')
                        # if i.going_DOWN(line_down,line_up)==True:
                        # if self.tracks[-1][1]>mid_start and self.tracks[-2][1]<=mid_start:
                        if old_y+old_h < line_up and y+h >= line_up: # going down
                            insertingFlag=True

                cv2.circle(frame,(cx,y+h),5,(0,0,255),-1)
                img=cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)

                old_x = x
                old_y = y
                old_w = w
                old_h = h
                old_cx = cx
                old_cy = cy                
        else:
            insertingFlag=False 

        if insertingFlag == True:
            status='STATUS: Inserting object.'
        elif insertingFlag == False:
            status='STATUS: No object.'
        # str_up='UP: '+str(cnt_up)
        # str_down='DOWN: '+str(cnt_down)
        # frame=cv2.polylines(frame,[pts_L1],False,line_down_color,thickness=2)
        frame=cv2.polylines(frame,[pts_L2],False,line_up_color,thickness=2)
        frame=cv2.polylines(frame,[pts_L3],False,(255,255,255),thickness=1)
        # frame=cv2.polylines(frame,[pts_L4],False,(255,255,255),thickness=1)
        # cv2.putText(frame, str_up, (10, 40), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        # cv2.putText(frame, str_up, (10, 40), font, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
        # cv2.putText(frame, str_down, (10, 90), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        # cv2.putText(frame, str_down, (10, 90), font, 0.5, (255, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(frame, status, (10, 90), font, 0.5, (255, 0, 0), 1, cv2.LINE_AA)
        cv2.imshow('Frame',frame)

    if cv2.waitKey(1)&0xff==ord('q'):
        break
# else:
    # break
cap.release()
# cv2.destroyAllWindows()


In [ ]:
# class try_match:
#     def __init__(self, frame):
#         # t0 = cv2.imread('../templates/0.png', cv2.IMREAD_GRAYSCALE)
#         # t1 = cv2.imread('../templates/1.png', cv2.IMREAD_GRAYSCALE)
#         # t2 = cv2.imread('../templates/2.png', cv2.IMREAD_GRAYSCALE)
#         # t3 = cv2.imread('../templates/3.png', cv2.IMREAD_GRAYSCALE)
#         # t4 = cv2.imread('../templates/4.png', cv2.IMREAD_GRAYSCALE)
#         # t5 = cv2.imread('../templates/5.png', cv2.IMREAD_GRAYSCALE)
#         # t6 = cv2.imread('../templates/6.png', cv2.IMREAD_GRAYSCALE)
#         self.tb = cv2.imread('templates/fb.png', cv2.IMREAD_GRAYSCALE)
#         self.frame = frame

        
#         # img3 = cv2.imread('/home/joaopaulo/Documents/UFC/Oitavo/Sis_Embarcados/JustTake/Visao/test/qrcode.jpg', cv2.IMREAD_GRAYSCALE)

#     def matching_Compute(self):
#         orb = cv2.ORB_create()
#         kp1, des1 = orb.detectAndCompute(self.tb, None)
#         kp2, des2 = orb.detectAndCompute(self.frame, None)

#         bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
#         matches=bf.match(des1,des2)
#         matches=sorted(matches, key=lambda x:x.distance)
#         matching_result = cv2.drawMatches(self.tb, kp1, self.frame, kp2, matches[:10], None)
#         print(matches[:10])
#         return matching_result


#         # cv2.imshow("template", matching_result)
#         # cv2.imwrite("../image1.jpg", matching_result)
#         # cv2.imshow("vera", img2)
#         # cv2.imshow("code", img3)
#         # cv2.waitKey(0)
#         # cv2.destroyAllWindows()

# # MIN_MATCH_COUNT = 10
# # # img1 = cv2.imread('box.png',0)          # queryImage
# # # img2 = cv2.imread('box_in_scene.png',0) # trainImage
# # # Initiate SIFT detector
# # sift = cv2.xfeatures2d.SIFT_create()
# # # find the keypoints and descriptors with SIFT
# # kp1, des1 = sift.detectAndCompute(img1,None)
# # kp2, des2 = sift.detectAndCompute(img2,None)
# # FLANN_INDEX_KDTREE = 1
# # index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
# # search_params = dict(checks = 50)
# # flann = cv2.FlannBasedMatcher(index_params, search_params)
# # matches = flann.knnMatch(des1,des2,k=2)
# # # store all the good matches as per Lowe's ratio test.
# # good = []
# # for m,n in matches:
# #     if m.distance < 0.7*n.distance:
# #         good.append(m)